In [19]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential


from azure.ai.ml.constants import AssetTypes

import json
import datetime
from dotenv import load_dotenv
import os



In [20]:
model_path = "../output/xgboost_model.pkl"

In [21]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()

In [22]:
# Load the content of the config.json file
with open('azure_config.json', 'r') as file:
    azure_config = json.load(file)

# enter details of your Azure Machine Learning workspace
subscription_id = azure_config.get('subscription_id')
resource_group = azure_config.get('resource_group')
workspace = azure_config.get('workspace_name')

# get a handle to the workspace
ml_client = MLClient(
    credential, subscription_id, resource_group, workspace
)

#### Register model

In [13]:
file_model = Model(
    path=model_path,
    type=AssetTypes.CUSTOM_MODEL,
    name="xgboost_model",
    description="XGBoost Model recommender",
)
ml_client.models.create_or_update(file_model)

Uploading xgboost_model.pkl (< 1 MB): 100%|##########| 171k/171k [00:00<00:00, 1.01MB/s]




Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'xgboost_model', 'description': 'XGBoost Model recommender', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/aa499b08-3aec-4b69-aec8-fe92d0cb1c99/resourceGroups/bobafy/providers/Microsoft.MachineLearningServices/workspaces/bobafy_model_dev/models/xgboost_model/versions/1', 'Resource__source_path': '', 'base_path': 'c:\\Users\\User\\Documents\\Ali\\Projects\\buildspace\\bbtreco\\model\\deploy', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x000002F30EE43590>, 'serialize': <msrest.serialization.Serializer object at 0x000002F30EF42120>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/aa499b08-3aec-4b69-aec8-fe92d0cb1c99/resourceGroups/bobafy/workspaces/bobafy_model_dev/datastores/workspaceblobstore/paths/LocalUpload/711d4142fab7a85cea2e56c588edb92d/xgboost_mod

#### Environment

In [35]:
env = Environment(
    conda_file="conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    name="bobafy-xgboost-env",
    description="Environment created from a Docker image plus Conda environment.",
)

ml_client.environments.create_or_update(env)

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'bobafy-xgboost-env', 'description': 'Environment created from a Docker image plus Conda environment.', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/aa499b08-3aec-4b69-aec8-fe92d0cb1c99/resourceGroups/bobafy/providers/Microsoft.MachineLearningServices/workspaces/bobafy_model_dev/environments/bobafy-xgboost-env/versions/4', 'Resource__source_path': '', 'base_path': 'c:\\Users\\User\\Documents\\Ali\\Projects\\buildspace\\bbtreco\\model\\deploy', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x00000254F6C04B30>, 'serialize': <msrest.serialization.Serializer object at 0x00000254F6C07740>, 'version': '4', 'conda_file': {'channels': ['defaults']

#### Endpoint

In [14]:
# Define an endpoint name
endpoint_name = "bobafy-endpoint"

endpoint_name = endpoint_name + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name = endpoint_name, 
    description="online endpoint for bobafy recommendation model",
    auth_mode="key"
)

print(endpoint_name)

bobafy-endpoint07222040497208


In [16]:
for endpoint in ml_client.online_endpoints.list():
    print(endpoint.name)

bobafy-endpoint07212348548964


In [ ]:
ml_client.online_endpoints.begin_create_or_update(endpoint)

#### Deployment

In [ ]:
# Deploy locally first
ml_client.online_endpoints.begin_create_or_update(endpoint, local=True)

In [48]:
# model = Model(path="model/output/xgboost_model.pkl")
# env = Environment(
#     conda_file="model/deploy/conda.yaml",
#     image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
# )


model = "azureml:xgboost_model:1"
env = "azureml:bobafy-xgboost-env:4"
endpoint_name = "bobafy-endpoint07212348548964"

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    # scoring_script = "score.py",
    code_configuration=CodeConfiguration(
        code=".", scoring_script="score.py"
    ),
    instance_type="Standard_DS2_v2",
    instance_count=1,
)

In [49]:
ml_client.online_deployments.begin_create_or_update(blue_deployment)

Instance type Standard_DS2_v2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/en-us/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint bobafy-endpoint07212348548964 exists
Uploading deploy (0.1 MBs): 100%|##########| 101685/101685 [00:00<00:00, 336804.43it/s]




..........................................

In [ ]:
# Debug

# Get logs for the specific deployment
logs = ml_client.online_deployments.get_logs(name="blue", endpoint_name="bobafy-endpoint07212348548964", lines=500)
print(logs)

In [54]:
for endpoint in ml_client.online_endpoints.list():
    print(endpoint.name)

bobafy-endpoint07212348548964


#### Check if its up

In [59]:
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name="blue",
    request_file="sample-request.json",
)

'[{"user_id": 5, "drink_id": 200, "fruity_user": 1, "milky_user": 0, "with_tea_user": 1, "refreshing_user": 1, "fragrant_user": 0, "adventurous_user": 1, "cold_user": 1, "fruity_drink": 1, "milky_drink": 0, "with_tea_drink": 1, "refreshing_drink": 1, "fragrant_drink": 0, "cold_drink": 1, "distance": 2.5, "popularity": 85, "score": 0.950219452381134}, {"user_id": 5, "drink_id": 202, "fruity_user": 1, "milky_user": 0, "with_tea_user": 1, "refreshing_user": 1, "fragrant_user": 0, "adventurous_user": 1, "cold_user": 1, "fruity_drink": 1, "milky_drink": 1, "with_tea_drink": 0, "refreshing_drink": 1, "fragrant_drink": 0, "cold_drink": 0, "distance": 1.5, "popularity": 75, "score": -0.139688178896904}, {"user_id": 5, "drink_id": 201, "fruity_user": 1, "milky_user": 0, "with_tea_user": 1, "refreshing_user": 1, "fragrant_user": 0, "adventurous_user": 1, "cold_user": 1, "fruity_drink": 0, "milky_drink": 1, "with_tea_drink": 1, "refreshing_drink": 0, "fragrant_drink": 1, "cold_drink": 1, "distanc